# Experiments

In [ ]:
import os
import pathlib
from dataclasses import dataclass
import tensorflow as tf
import pandas as pd
from dotenv import load_dotenv
import wandb

from bcd import DataClass
from bcd.model.network.aknet import AKNetConfig, AKNetFactory
from bcd.model.network.tmnet import TMNetConfig, TMNetFactory
from bcd.model.network.shainnet import ShainNetConfig, ShainNetFactory
from bcd.model.repo import ModelRepo
from bcd.model.base import DenseNet, ResNet, EfficientNet, Inception, InceptionResNet, Xception, MobileNet
from bcd.model.experiment import TransferLearningExperiment
from bcd.model.callback import Log
pd.set_option('display.max_rows',999)
load_dotenv()

## Configuration

In [ ]:
# Project Parameters
%env "WANDB_NOTEBOOK_NAME" "experiments.ipynb"
datasets = {"Development":{"name": "CBIS-DDSM_10","directory": "data/image/1_final/training_10/training/"},
            "Stage": {"name": "CBIS-DDSM_30","directory": "data/image/1_final/training_30/training/"},
            "Production": {"name": "CBIS-DDSM","directory": "data/image/1_final/training/training/"},
            }
mode = "Development"
project = f"Breast-Cancer-Detection-{mode}" 

# Experiment Parameters
force = True  # Whether to retrain if the model and config already exists.


# Training Config
metrics = ['accuracy', tf.keras.metrics.AUC(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
loss = "binary_crossentropy"
epochs = 100 # Maximum number of epochs to train, subject to any early stopping callback.
learning_rate = 1e-4,
training_config = {"loss": loss, "epochs": epochs, "learning_rate": learning_rate}

# Network Config
activation = "sigmoid" # Network configuration common to all networks.
    
# Fine Tuning Config
ft_fine_tune = False
ft_iceblocks = 10 # Number of blocks of layers to potentially thaw.
ft_sessions= 5  # Number of iceblocks to actually unthaw
ft_learning_rate_init = 1e-5 # Learning rate at the beginning of fine tuning.
ft_learning_rate_end = 1e-8 # The lowest learning rate subject to learning rate decay during fine tuning.
ft_epochs_init: int = 5  # Number of epochs to train first session
ft_epochs_end: int = 50  # Number of epochs to train last session.
fine_tune_config = {"fine_tune": ft_fine_tune, "iceblocks": ft_iceblocks, "sessions": ft_sessions, 
                    "learning_rate_init": ft_learning_rate_init, 
                    "learning_rate_end": ft_learning_rate_end,
                    "epochs_init": ft_epochs_init,
                    "epochs_end": ft_epochs_end,
                    }

# Dataset params
dataset = datasets[mode]["name"]
batch_size = 64 if mode == "Production" else 32
input_shape = (224,224,3)
output_shape = 1
train_dir = pathlib.Path(datasets[mode]["directory"]).with_suffix('') 
dataset_config = {"dataset": dataset, "batch_size": batch_size, "input_shape": input_shape, "output_shape": output_shape}

# Checkpoint Config
ckpt_monitor = "val_accuracy"
ckpt_verbose = 1
ckpt_save_best_only = True
ckpt_save_weights_only = False
ckpt_mode = "auto"
checkpoint_config = {"monitor": ckpt_monitor, "verbose": ckpt_verbose, "save_best_only": ckpt_save_best_only, "mode": ckpt_mode}

# Early stop parameters 
es_min_delta = 0.0001
es_monitor = "val_loss"  # Monitor validation loss for early stopping
es_patience = 10  # The number of epochs for which lack of improvement is tolerated 
es_restore_best_weights = True  # Returns the best weights rather than the weights at the last epoch.
es_verbose = 1
early_stop_config = {"min_delta": es_min_delta, "monitor": es_monitor, "patience": es_patience, 
                     "restore_best_weights": es_restore_best_weights, "verbose": es_verbose}

# Reduce LR on Plateau Parameters
rlr_monitor = "val_loss"
rlr_factor = 0.5
rlr_patience = 3
rlr_verbose = 1
rlr_mode = "auto"
rlr_min_delta = 1e-4
rlr_min_lr=1e-10
learning_rate_config = {"monitor": rlr_monitor, "factor": rlr_factor, "patience": rlr_patience, "verbose": 
                        rlr_verbose, "mode": rlr_mode, "min_delta": rlr_min_delta, "min_lr": rlr_min_lr}



## Experiment Config

In [ ]:
config = {
    "project": project,       
    "mode": mode,
    "dataset_config": dataset_config,
    "fine_tune_config": fine_tune_config,
    "training_config": training_config,
    "checkpoint_config": checkpoint_config,
    "early_stop_config": early_stop_config,
    "learning_rate_config": learning_rate_config,
    
}

## Load Data

In [ ]:
# Training DataSet (10%)
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels="inferred",
    color_mode="rgb",
    image_size=(224,224),
    shuffle=True,
    validation_split=0.2,
    subset='training',
    interpolation="bilinear",
    seed=123,
    batch_size=batch_size)

# Validation DataSet (10%)
val_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels="inferred",
    color_mode="rgb",
    image_size=(224,224),
    shuffle=True,
    validation_split=0.2,
    subset='validation',
    interpolation="bilinear",
    seed=123,
    batch_size=batch_size)

In [ ]:
val_ds.class_names

## Callbacks

In [ ]:
early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor=es_monitor, 
                                                       min_delta=es_min_delta,
                                                       patience=es_patience, 
                                                       restore_best_weights=es_restore_best_weights,
                                                       verbose=es_verbose)

reduce_lr_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor=rlr_monitor,
                                                          factor=rlr_factor,
                                                          patience=rlr_patience,
                                                          verbose=rlr_verbose,
                                                          mode=rlr_mode,
                                                          min_delta=rlr_min_delta,
                                                          min_lr=rlr_min_lr)
callbacks = [early_stop_callback, reduce_lr_callback]

## Dependencies

In [ ]:
repo = ModelRepo(mode = mode)
optimizer = 

## ShainNet 

In [ ]:
tmnet_config = ShainNetConfig(activation=activation)
factory = TMNetFactory(config=tmnet_config, input_shape=input_shape, output_shape=output_shape, activation=activation)
densenet = factory.create(base_model=DenseNet())
efficientnet = factory.create(base_model=EfficientNet())
inception = factory.create(base_model=Inception())
inception_resnet = factory.create(base_model=InceptionResNet())
mobilenet = factory.create(base_model=MobileNet())
resnet = factory.create(base_model=ResNet())
xception = factory.create(base_model=Xception())
networks = [densenet, efficientnet, inception, inception_resnet, mobilenet, resnet, xception]

## ShainNet
Instantiate and run the project.

In [9]:
experiment = TransferLearningExperiment(networks=networks, config=config, repo=repo, callbacks=callbacks, metrics=metrics, optimizer=optimizer, force=force)
experiment.run(train_ds=train_ds, val_ds=val_ds)

7/7 [==============================] - ETA: 0s - loss: 0.9280 - accuracy: 0.4887 - auc: 0.5867 - precision: 0.4420 - recall: 0.8696
Epoch 4: val_accuracy did not improve from 0.70909

Epoch 4: val_accuracy did not improve from 0.65455

Epoch 4: val_accuracy did not improve from 0.60000

Epoch 4: val_accuracy did not improve from 0.60000

Epoch 4: val_accuracy did not improve from 0.60000

Epoch 4: val_accuracy did not improve from 0.60000

Epoch 4: val_accuracy did not improve from 0.36364
7/7 [==============================] - 32s 4s/step - loss: 0.9280 - accuracy: 0.4887 - auc: 0.5867 - precision: 0.4420 - recall: 0.8696 - val_loss: 0.7394 - val_accuracy: 0.3455 - val_auc: 0.4876 - val_precision: 0.3137 - val_recall: 0.9412 - lr: 3.8147e-10
Epoch 5/100
5/7 [====================>.........] - ETA: 5s - loss: 1.0147 - accuracy: 0.4500 - auc: 0.4590 - precision: 0.4370 - recall: 0.8310

KeyboardInterrupt: 